## 04 Model Training

#### Import relevant libraries

In [11]:
import os
import sklearn as skl
import pandas as pd
import numpy as np

#### Load model training data

In [12]:
input_dir = os.path.join('..', 'data', 'processed')
df = pd.read_csv(os.path.join(input_dir, 'processed_data.csv'))

/var/folders/c9/qj7_h93n2zqdn4_67hfnjgkc0000gn/T/ipykernel_14424/2736453946.py:2: DtypeWarning: Columns (5,6,7,8,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(input_dir, 'processed_data.csv'))


In [ ]:
# Creating arrays and features for the target variable (expenditure / sales)

# 'Normalized Cost of Energy' is a placeholder for the column name with the target variable
target = 'Normalized Cost of Energy'

y = df[target]
x = df.drop(columns = [target])

#### Scale predictor variables

In [ ]:
# Import scale from sklearn
from sklearn.preprocessing import scale

# A scaling function that does not affect categorical variables such as respondent id,
# year, or regulation
def scale_function(X):
    x_scaled = X
    x_scaled = x_scaled.drop(columns = ['respondent_id', 'year'])
    x_scaled = scale(x_scaled)
    x_scaled['respondent_id'] = X['respondent_id']
    x_scaled['year'] = X['year']
    return x_scaled

In [ ]:
# Scale variables - use scale_function(X) if we use categorical variables
X_scaled = scale_function(X)

#### Split into test and train sets

In [9]:
# Import train_test_split from sklearn
from sklearn.model_selection import train_test_split

# Create training and test sets (not quite sure how to use test_size and random_state)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=None, random_state=None)


#### Fit regression model

(a) Linear regression

In [ ]:
# Import LinearRegression from sklearn
from sklearn.linear_model import LinearRegression

# Instantiate the regressor
reg_lin = LinearRegression()

# Fit the regressor to the training data
reg_lin.fit(X_train, y_train)

(b) Random forest

In [10]:
# Import RandomForestRegressor from sklearn
from sklearn.ensemble import RandomForestRegressor

# Instantiate the regressor with default value for hyperparameters
reg_forest = RandomForestRegressor(random_state=None)

# Fit the regressor to the training data
reg_forest.fit(X_train, y_train)

NameError: name 'X_train' is not defined

(c) Neural Network

In [ ]:
# Import MLPRegressor from skleatn
from sklearn.neural_network import MLPRegressor

#Instantiate the regressor with default values
reg_neural = MLPRegressor(random_state=None)

# Fit the regressor to the training data
reg_neural.fit(X_train, y_train)

#### Pickle model_fit file(s)

In [ ]:
# Save linear regression model to disk
filename = 'lin_model_1.sav'
pickle.dump(reg_lin, open(filename, 'wb'))

In [ ]:
# Save random forest model to disk
filename = 'forest_model_1.sav'
pickle.dump(reg_forest, open(filename, 'wb'))

In [ ]:
# Save neural network model to disk
filename = 'neural_model_1.sav'
pickle.dump(reg_neural, open(filename, 'wb'))